In [ ]:
import pandas as pd
import numpy as np
import logging as log

from data_prepare import getLogsFromDBByUserID

In [ ]:
df = pd.read_pickle('../data/173211-9739.pkl')
df['event'] = df['method'] + ':' + df['urlEntry']

In [ ]:
eventMap = {event: id for id, event in enumerate(df['event'].value_counts().keys())}
for id in eventMap.values(): 
    if id == 0:
        print(id)

In [ ]:
for id, event in enumerate(df['event'].value_counts().keys()):
    print((id, event))

In [ ]:
from datetime import datetime
# 增加日期
df['date'] = df['time'].apply(lambda x : datetime.fromtimestamp(x))
df.sort_values(by='time', ascending=True, inplace=True)
df = df.reset_index(drop=True)

In [ ]:
dates = df['date'].copy()
dates.sort_values(ascending=True, inplace=True)
# print(dates)
values = []
for i in range(1, len(dates)):
    values.append((dates[i] - dates[i-1]).total_seconds())
    if (dates[i] < dates[i-1]): 
        print(i, dates[i-1], dates[i])
v = np.asarray(values)
# values

In [ ]:
dates

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# threshold = 60*60*2
threshlodd = 30
data = []
for i in values:
#     if i < threshold and i >= threshlodd: 
    data.append(i)
# max(data)
# d = np.asarray(data)
# plt.hist(bins=300, x=d)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np


In [ ]:
months = df['date'].apply(lambda x : "%d-%d"%(x.year, x.month))
data = months.value_counts()
data.plot()

In [ ]:
df

In [ ]:
output = '../data/rewriteLog.log'
with open(output, 'w') as f:
    for logEntry in df.itertuples():
        f.write("%s %s %s %s %s %s %s\n" % (
            logEntry[8],
            logEntry[2],
            logEntry[3],
            logEntry[4],
            logEntry[5],
            logEntry[6],
            logEntry[7],
        ))

In [ ]:
from spellpy import spell

tau = 0.5
# input_dir = '../data/'
output_dir = '../data/output/'
log_format = '<Date> <Time> <Url> <Method> <Content> <Headers> <Name> <ADDR>'

parser = spell.LogParser(
    indir='./',
    outdir='./',
    log_format=log_format,
#     logmain=log_main,
    tau=tau,
)

parser.parse(output)


In [ ]:
df

In [ ]:
def deeplog_df_transfer(df, event_id_map):
  
    df = df[['date', 'event']]
    df['EventId'] = df['event'].apply(
        lambda e: event_id_map[e] if event_id_map.get(e) else -1)
    deeplog_df = df.set_index('date').resample(
        '30min').apply(_custom_resampler).reset_index()
    return deeplog_df


def _custom_resampler(array_like):
    return list(array_like)

def deeplog_file_generator(filename, df):
    with open(filename, 'w') as f:
        for event_id_list in df['EventId']:
            if (len(event_id_list) == 0):
                continue
            for event_id in event_id_list:
                f.write(str(event_id) + ' ')
            f.write('\n')

In [ ]:
df['event'] = df['method'] + ':' + df['urlEntry']
event_id_map = dict()
for i, event_id in enumerate(df['event'].unique(), 1):
    event_id_map[event_id] = i
event_id_map

## train

In [ ]:
epochs = range (1, 5)
losss = [0.015, 0.025, 0.045, 0.065]
accs = [0.085, 0.075, 0.045, 0.015]

import matplotlib.pyplot as plt
fig = plt.figure()

ax1 = fig.add_subplot(2, 1, 1)
ax2 = fig.add_subplot(2, 1, 2)
ax1.plot(epochs, losss, 'k')
ax2.plot(accs, losss)


In [1]:
urls = []
with open("../data/urls_origin.txt", 'r') as f:
    for line in f:
        urls.append(line.strip())
urls.__len__()

275974

In [2]:
from modules.url_match import UrlMatcher, UrlEntry
from sql.db import DB as db

entrys = db.queryUrlEntrys()

# entrys
# matcher = UrlMatcher()

In [3]:
urlEntrys = [UrlEntry(paraSet=(entry[2].split('/')[1], entry[1], entry[2])) for entry in entrys]
matcher = UrlMatcher(urlEntrys)

In [5]:
dic = set()
with open('../data/all_urls.csv', 'r') as f:
    first = True
    for line in f:
        if first:
            first = False
            continue
        ss = line.strip().split(',')
        module = ss[-1]
        url = line.strip().strip(','+module)
        url = '/' + url.strip('/')
        try:
            entry = matcher.convert(url, module)
            dic.add(entry)    
        except Exception as e:
            print(e)
            

In [11]:
# '/hr/thrpeople' in dic
matcher.convert('/hr/tpayusers', 'POST')

In [15]:
import pandas as pd
from datetime import datetime
data = [
    [1629089285.177, 1],
    [1629089385.177, 2],
    [1629089485.177, 3],
    [1629089585.177, 4],
    [1629089685.177, 5],
    [1639470245, 6],
]

df = pd.DataFrame(data, columns=['time', 'label'])

df['date'] = df['time'].apply(
            lambda timestamp: datetime.fromtimestamp(timestamp))
# df['event'] = df['method'] + ':' + df['urlEntry']
tdf = df[['date', 'label']]
tdf

,date,label
0,2021-08-16 00:48:05.177,1
1,2021-08-16 00:49:45.177,2
2,2021-08-16 00:51:25.177,3
3,2021-08-16 00:53:05.177,4
4,2021-08-16 00:54:45.177,5
5,2021-12-14 03:24:05.000,6


In [45]:
import numpy as np
# tdf.set_index('date').resample('3min').apply(
#             lambda array: list(array)).reset_index()
ttdf = tdf.set_index('date').resample('40min').apply(
    lambda array: list(array)
).reset_index()

def maxCount(values):
    if len(values) <= 1:
        return values
    return np.argmax(np.bincount(values))

# ttdf
ttdf['label'].apply(lambda values : np.argmax(np.bincount(values)) if len(values) >= 1 else 0)

# ttdf['label']
# tttdf = ttdf['label'].dropna(0)
# tttdf

0       1
1       0
2       0
3       0
4       0
       ..
4320    0
4321    0
4322    0
4323    0
4324    6
Name: label, Length: 4325, dtype: int64

In [3]:
from typing import List
from sql.db import DB as db
from sql.db import ORIGIN_TABLE_LABELS
import pandas as pd

def getLogsFromDBByUserID(userID: str, columns: List[str] = []) -> pd.DataFrame:
    """
    getLogsFromDBByDate 获取某一时间段的日志，可支持获取日志的某几个字段

    Args:
        userID (str): userID
        columns (List[str]): 想要获取的字段，如果此项为空，则默认获取全部字段
    Returns:
        pd.DataFrame: 返回的日志数据

    ```
    userID = '173211-9739'
    df = getLogsFromDBByUserID(
        userID, ['time', 'urlEntry', 'method', 'parameterName', 'headers', 'name', 'ip'])
    ```
    df.to_pickle('data/%s.pkl' % userID)
    """
    if (columns == None or len(columns) == 0):
        columns = ORIGIN_TABLE_LABELS

    equalLabel = 'userID'

    log.debug("获取日志字段 %s, 要求 userID 为 %s " % (','.join(columns),  userID))

    originLogs = db.queryOriginWithEqual(columns, equalLabel, userID)
    log.debug("获取到数据%d条，columns为%s" % (len(originLogs), ','.join(columns)))
    df = pd.DataFrame(originLogs, columns=columns)
    # log.debug(df)
    return df

In [11]:
import numpy as np
import pandas as pd
from datetime import datetime
import logging as log 
# from data_prepare import getLogsFromDBByUserID
# import data_prepare

users = ['173211-1000847', '173211-1000852']
columns = [
    'time', 'urlEntry', 'method'
]

ths = '40min'

eventMap = {}
user = users[0]
# for id, user in enumerate(users):
log.debug("开始处理用户%s, %d/%d" % (user, 1, 1))
tdf = getLogsFromDBByUserID(user, columns)
log.debug("共获取到%d条数据" % len(tdf))

tdf['date'] = tdf['time'].apply(
    lambda timestamp: datetime.fromtimestamp(timestamp))
tdf['event'] = tdf['method'] + ':' + tdf['urlEntry']
tdf = tdf[['date', 'event']]

# tdf['userId'] = 
# # update event map
# # eventMap = {event: id for id, event in enumerate(
# #     df['event'].value_counts().keys())}
for event in tdf['event']:
    if event not in eventMap:
        eventMap[event] = len(eventMap)

tdf['eventId'] = tdf['event'].apply(lambda e: eventMap[e])
# tdf['userId'] = user

rdf = tdf.set_index('date').resample(ths).apply(
    lambda array: list(array)).reset_index()
rdf = rdf.dropna(axis=0, subset=['eventId'])
rdf['userId'] = user
rdf
# df = pd.concat([df, rdf], ignore_index=True)
# # 制作长序列
# lseq = []
# for seq in rdf['eventId']:
#     cnter = collections.Counter(seq)
#     lseq.extend([i for i, _ in cnter.most_common(2)])
# longSeq = longSeq.append(' '.join(lseq))

,date,event,eventId,userId
0,2020-04-03 09:20:00,[GET:/fa/commoncore/assetCommonCore/remoteGetC...,"[0, 1, 2]",173211-1000847
1,2020-04-03 10:00:00,[],[],173211-1000847
2,2020-04-03 10:40:00,[],[],173211-1000847
3,2020-04-03 11:20:00,[],[],173211-1000847
4,2020-04-03 12:00:00,[],[],173211-1000847
...,...,...,...,...
12856,2021-03-26 12:00:00,[],[],173211-1000847
12857,2021-03-26 12:40:00,[],[],173211-1000847
12858,2021-03-26 13:20:00,[],[],173211-1000847
12859,2021-03-26 14:00:00,"[GET:/portal/menus/mine/pc, GET:/portal/shortc...","[12, 13, 14, 5, 0, 6, 13, 1, 14, 9, 10, 8, 7, ...",173211-1000847


In [8]:
tdf['userId'] = user
tdf

,date,event,eventId,userId
0,2020-04-03 09:37:42.778,GET:/fa/commoncore/assetCommonCore/remoteGetCu...,0,173211-1000847
1,2020-04-03 09:37:42.874,GET:/fa/asset/assetInfo/remoteGetPagedCanUseList,1,173211-1000847
2,2020-04-03 09:37:43.770,GET:/fa/commoncore/todoCommonCore/remoteGetTot...,2,173211-1000847
3,2020-04-03 16:23:48.044,GET:/fa/commoncore/assetCommonCore/remoteGetCu...,0,173211-1000847
4,2020-04-03 16:23:48.095,GET:/fa/asset/assetInfo/remoteGetPagedCanUseList,1,173211-1000847
5,2020-04-03 16:23:48.846,GET:/fa/commoncore/todoCommonCore/remoteGetTot...,2,173211-1000847
6,2020-04-03 16:23:49.926,GET:/fa/commoncore/assetCommonCore/remoteGetDa...,3,173211-1000847
7,2020-04-03 16:23:49.949,GET:/fa/commoncore/todoCommonCore/remoteGetPag...,4,173211-1000847
8,2020-04-03 16:23:52.644,GET:/fa/commoncore/assetCommonCore/remoteGetCu...,0,173211-1000847
9,2020-04-03 16:23:52.683,GET:/fa/asset/assetInfo/remoteGetPagedCanUseList,1,173211-1000847


In [18]:
basePath = './'
trainSize = 5
validationSize = 2
testSize = 3
df = rdf

# df.iloc[1,]['date']

lines = []
dic = set()
# lines = set()
for id in range(len(df)):
    data = df.iloc[id,]
    if (len(data['eventId']) <= 5):
        continue
    print(data)
    lines.append('{} {} {}'.format(
        data['date'], 
        data['userId'],
        ' '.join([str(i) for i in data['eventId']])
    ))
lines
#     lines.add('{} {} {}'.format(
#         ,
#         ,
#         ,' '.join([str(id) for id in event_id_list]))
#     )
# lines = list(lines)
# mkdirIfNotExist(basePath)
# all = trainSize + validationSize + testSize
# length = len(lines)
# flag1 = int(length * trainSize / all)
# flag2 = int(length * (trainSize+validationSize) / all)
# log.info("去重后，共得到序列 {} 条, 训练集 {} 条，验证集 {} 条，测试集 {} 条".format(
#     length, flag1, flag2-flag1, length-flag2))

# writeList(basePath + '/train', lines[:flag1])
# writeList(basePath + '/validation', lines[flag1:flag2])
# writeList(basePath + '/test', lines[flag2:])

date                                     2020-04-03 16:00:00
event      [GET:/fa/commoncore/assetCommonCore/remoteGetC...
eventId                             [0, 1, 2, 3, 4, 0, 1, 2]
userId                                        173211-1000847
Name: 10, dtype: object
date                                     2021-03-26 14:00:00
event      [GET:/portal/menus/mine/pc, GET:/portal/shortc...
eventId    [12, 13, 14, 5, 0, 6, 13, 1, 14, 9, 10, 8, 7, ...
userId                                        173211-1000847
Name: 12859, dtype: object
date                                     2021-03-26 14:40:00
event      [GET:/pm/integrate/prjInfoIntegrate/getPersona...
eventId             [10, 0, 5, 6, 1, 9, 8, 13, 14, 7, 11, 2]
userId                                        173211-1000847
Name: 12860, dtype: object


['2020-04-03 16:00:00 173211-1000847 0 1 2 3 4 0 1 2',
 '2021-03-26 14:00:00 173211-1000847 12 13 14 5 0 6 13 1 14 9 10 8 7 11 2 15 12 15 12 16 13 14 17 5 0 10 6 9 13 14 1 8 7 11 2 17',
 '2021-03-26 14:40:00 173211-1000847 10 0 5 6 1 9 8 13 14 7 11 2']

In [97]:
dDf = tdf.set_index('date').resample("40min").apply(
    lambda array: list(array) if len(array) > 0 else None).reset_index()

In [98]:
rdf = dDf.dropna(axis=0, subset=['EventId'])

In [99]:
rdf['len'] =rdf['EventId'].apply(lambda array : len( array ))
rdf

/home/chase/miniconda3/envs/deeplog/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,date,event,EventId,len
0,2020-10-14 10:40:00,"[GET:/portal/menus/mine/pc, GET:/portal/shortc...","[20, 21, 22, 4, 21, 5, 0, 15, 22, 1, 8, 17, 6,...",51
6,2020-10-14 14:40:00,[GET:/pm/integrate/prjInfoIntegrate/getPersona...,"[17, 0, 5, 4, 1, 21, 22, 15, 8, 6, 18, 2]",12
225,2020-10-20 16:40:00,[GET:/portal/shortcutconfig/getConfigCompletio...,"[21, 20, 22, 0, 4, 5, 21, 15, 8, 17, 22, 1, 6,...",28
683,2020-11-02 10:00:00,"[GET:/portal/menus/mine/pc, GET:/portal/shortc...","[20, 21, 22, 4, 5, 0, 15, 21, 8, 17, 22, 1, 6,...",28
10061,2021-07-20 22:00:00,[GET:/fin/ar/billSource/getNodeDetail/{sourceI...,"[4, 0, 5, 14, 3, 19, 16, 17, 1, 8, 7, 8, 10, 1...",45


In [2]:
import collections

for line in rdf['EventId']:
    # print(line)
    i = collections.Counter(line)
    # print(i.most_common(2).keys)
    # seq = [c for c, _ in i.most_common(2).keys]
    # print(seq)
longdf = pd.DataFrame(columns=['user', 'seq'])
longdf
lseq = []
user = 'ajfia'
# s = pd.Series(data = user, name='user')
# seq = pd.Series(data=lseq, name='seq')
# d = pd.DataFrame([s, seq]).reset_index()
# d
for seq in rdf['EventId']:
    cnter = collections.Counter(seq)
    lseq.extend([i for i, _ in cnter.most_common(2)])
# # print(lseq)
# t = pd.DataFrame(
#         data=[[user], [lseq]], 
#         # columns=['userId', 'seq']
#     )
# t
t
# longdf = pd.concat([
#     longdf, 
#     t], ignore_index=True)
# longdf   
#     ignore_index=True
# )
# longdf

NameError: name 'rdf' is not defined

In [5]:
import logging as log 
user = '173211-21256'
id = 1
from datetime import datetime
import pandas as pd
eventMap = {}
import collections
ths = '40min'
userNum = 2
columns = [
    'time', 'urlEntry', 'method'
]
log.debug("开始处理用户%s, %d/%d" % (user, id, userNum))
tdf = getLogsFromDBByUserID(user, columns)
log.debug("共获取到%d条数据" % len(tdf))
df = pd.DataFrame()
longSeq = pd.DataFrame()
tdf['date'] = tdf['time'].apply(
    lambda timestamp: datetime.fromtimestamp(timestamp))
tdf['event'] = tdf['method'] + ':' + tdf['urlEntry']
tdf = tdf[['date', 'event']]
# update event map
# eventMap = {event: id for id, event in enumerate(
#     df['event'].value_counts().keys())}
for event in tdf['event']:
    if event not in eventMap:
        eventMap[event] = len(eventMap)

tdf['eventId'] = tdf['event'].apply(lambda e: eventMap[e])
rdf = tdf.set_index('date').resample(ths).apply(
    lambda array: list(array)).reset_index()
rdf = rdf.dropna(axis=0, subset=['eventId'])
rdf['userId'] = user
df = pd.concat([df, rdf], ignore_index=True)
# 制作长序列
lseq = []
for seq in rdf['eventId']:
    cnter = collections.Counter(seq)
    lseq.extend([str(i) for i, _ in cnter.most_common(2)])


In [20]:
lseqdf = pd.DataFrame({'eventId': [lseq]})
lseqdf['userId'] = user
lseqdf['date'] = tdf.iloc[0, ]['date']
lseqdf
# lseq
# df['eventId']

,eventId,userId,date
0,0,173211-21256,2020-04-01 10:02:09.185
1,1,173211-21256,2020-04-01 10:02:09.185
2,0,173211-21256,2020-04-01 10:02:09.185
3,1,173211-21256,2020-04-01 10:02:09.185
4,0,173211-21256,2020-04-01 10:02:09.185
...,...,...,...
1013,91,173211-21256,2020-04-01 10:02:09.185
1014,98,173211-21256,2020-04-01 10:02:09.185
1015,97,173211-21256,2020-04-01 10:02:09.185
1016,51,173211-21256,2020-04-01 10:02:09.185


In [16]:
df['eventId']
import numpy as np 


0                                                [0, 1, 2]
1                                                       []
2                                                       []
3                                                       []
4                                                       []
                               ...                        
17313                                                   []
17314                                                   []
17315                                                   []
17316                                                   []
17317    [90, 0, 91, 206, 194, 273, 199, 203, 1, 89, 19...
Name: eventId, Length: 17318, dtype: object